# COPT and LOLP Calculator

## import and settting printer

In [ ]:
import itertools
import numpy as np
import pandas as pd

In [ ]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [ ]:
# # NOTE:
# #   UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
# #   COMMENT IF USING MANUAL INPUT
# # csv input path
# df = pd.read_csv('../data/COPT Case.csv')

# # extract csv
# capacities = df['Capacity (MW)'].tolist()
# outage_rates = df['FOR (Outage)'].tolist()
# status = [True] * len(df.index)

In [ ]:
# NOTE:
#   UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
#   COMMENT IF USING CSV INPUT
# max capacity
capacities = [
    80,
    70,
    40,
    50,
    40,
    50
]

# forced outage rate
outage_rates = [
    0.1, 
    0.2,
    0.1,
    0.2,
    0.1,
    0.1
]


# status of generator,
#     True: exist
#     False: not exist
status = [
    True,
    False,
    True,
    True,
    True,
    True,
]

In [ ]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,80,0.100,True
2,70,0.200,False
3,40,0.100,True
4,50,0.200,True
5,40,0.100,True
6,50,0.100,True


In [ ]:
# peak load or load at time considering
# used to calculate LOLP
demands = [190, 170, 185, 150, 70]

## COPT table

In [ ]:
BATCH_SIZE = 10
MIN_COVERAGE = 0.80

In [ ]:
ls = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

capacities_combination_generator = itertools.product(*capacities_fix)
probabilities_combinations_generator = itertools.product(*probabilities_fix)

capacity_merged = []
probability_merged = []

while sum(probability_merged) <= MIN_COVERAGE:
    capacity_merged += [sum(i) for i in itertools.islice(capacities_combination_generator, BATCH_SIZE)]
    probability_merged += [np.prod(i) for i in itertools.islice(probabilities_combinations_generator, BATCH_SIZE)]

# combine capacity and individual probability
table = np.vstack([capacity_merged, probability_merged]).T

# sort the combined table
table = table[(-table[:, 0]).argsort(),:]

# combine duplicate
table = np.array([[k,sum([x[1] for x in list(g)])] 
                 for k,g in itertools.groupby(table, lambda x:x[0])])

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

In [ ]:
columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,260.000,0.525,0.866,0.525
2,220.000,0.117,0.341,0.642
3,210.000,0.190,0.224,0.831
4,180.000,0.006,0.035,0.838
5,170.000,0.028,0.028,0.865
6,130.000,0.001,0.001,0.866


## LOLP

In [ ]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [ ]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
lolp

[0.03474000000000001,
 0.0007200000000000002,
 0.03474000000000001,
 0.0007200000000000002,
 0.0007200000000000002]

In [ ]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.0709
